## Raspagem de dados da Imobiliária X de Fernandópolis

In [1]:
import bs4 
import urllib.request as urllib_request
import pandas as pd

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError

url = 'x'
response = urlopen(url)
html = response.read().decode('utf-8')

### Rotina para pegar todos os anúncios em uma página

In [4]:
cards = []

In [96]:
url = 'x'
response = urlopen(url)
html = response.read().decode('utf-8')

soup = BeautifulSoup(html, 'html.parser') # criando objeto BeautifulSoup
anuncios = soup.findAll('span', {'class': 'thumb-info thumb-info-lighten'})

In [9]:
for i in range(8, 17):
    #Obtendo o HTML
    response = urlopen('x' + str(i + 1))
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
    # Obtendo as TAGs de interesse
    anuncios = soup.findAll('span', {'class': 'thumb-info thumb-info-lighten'})
    for anuncio in anuncios:
        card = {}

        card['id'] = int(anuncio.find('span', {'class': 'thumb-info-inner text-md font-weight-bold'}).getText().strip().split(' - ')[0])
        flag = anuncio.find('span', {'class': 'thumb-info-price fundo2 text-color-light text-md p-sm pl-md pr-md'})
        if(flag != None):
            card['preco'] = anuncio.find('span', {'class': 'thumb-info-price fundo2 text-color-light text-md p-sm pl-md pr-md'}).getText().strip().split('R$ ')[-1]
            card['preco'] = card['preco'].replace('.', '').replace(',','.')
        else:
            card['preco'] = 'NA'

        card['cidade'] = anuncio.find('span', {'class': 'thumb-info-inner text-md'}).getText()
        card['cidade'] = "".join(card['cidade'].split())

        card['cidade'] = anuncio.find('span', {'class': 'thumb-info-inner text-md'}).getText()
        card['cidade'] = card['cidade'].split(',')
        card['bairro'] = card['cidade'][0].strip().title()
        card['cidade'] = card['cidade'][-1].split('-')[0].strip().capitalize()

        aux = anuncio.find('ul',
                           {'class': 'accommodations font-weight-bold p-none text-sm'
                           }).findAll('li')
        if aux == []:
            card['bed'] = 'NA'
            card['bath'] = 'NA'
            card['car'] = 'NA'
        else:
            comodos = anuncio.find('ul',
                                   {'class': 'accommodations font-weight-bold p-none text-sm'
                                   }).findAll('span')
            tipo_comodo = anuncio.find('ul',
                                       {'class': 'accommodations font-weight-bold p-none text-sm'
                                       }).findAll('i')
            tipo_comodo = [tipo_comodo.get('class') for tipo_comodo in tipo_comodo]

            num_comodo = []

            for comodo in comodos:
                comodo = comodo.getText()
                comodo = ' '.join(str(comodo).strip())
                comodo = comodo.replace(' ', '')
                posicao = comodo.find('m')
                if posicao != -1:
                    comodo = comodo[0:posicao]
                elif comodo != '':
                    num_comodo.append(int(comodo))

            lista = []
            for nome_comodo in tipo_comodo:
                aux = (nome_comodo[1])[3:]
                if aux != 'map-marker' and aux != 'home':
                    lista.append(aux)

            comodos = {}
            for i in range(len(num_comodo)):
                comodos.update({lista[i]: num_comodo[i]})

            for value in comodos:
                card[value] = int(comodos.get(value))

            #card['comodos'] = comodos

        cards.append(card)

In [10]:
cards

[{'id': 835,
  'preco': '490000.00',
  'cidade': 'Fernandópolis',
  'bairro': 'Por Do Sol',
  'bed': 'NA',
  'bath': 'NA',
  'car': 'NA'},
 {'id': 834,
  'preco': '180000.00',
  'cidade': 'Fernandópolis',
  'bairro': 'Cohab',
  'bed': 'NA',
  'bath': 'NA',
  'car': 'NA'},
 {'id': 831,
  'preco': '250000.00',
  'cidade': 'Fernandópolis',
  'bairro': 'Rosa Amarela',
  'bed': 3,
  'bath': 3,
  'car': 2},
 {'id': 830,
  'preco': '180000.00',
  'cidade': 'Fernandópolis',
  'bairro': 'Cohab',
  'bed': 3,
  'bath': 3,
  'car': 2},
 {'id': 828,
  'preco': '140000.00',
  'cidade': 'Fernandópolis',
  'bairro': 'Parque Das Naçoes',
  'bed': 3,
  'bath': 1,
  'car': 2},
 {'id': 826,
  'preco': '265000.00',
  'cidade': 'Fernandópolis',
  'bairro': 'Fernandópolis-Sp',
  'bed': 3,
  'bath': 2,
  'car': 2},
 {'id': 825,
  'preco': 'NA',
  'cidade': 'Fernandópolis',
  'bairro': 'Mario Benez',
  'bed': 3,
  'bath': 2,
  'car': 2},
 {'id': 822,
  'preco': '230000.00',
  'cidade': 'Fernandópolis',
  'bair

### Criando dataframe

In [19]:
dataset = pd.DataFrame(cards)

In [20]:
dataset.head(10)

,id,preco,cidade,bairro,bed,bath,car
0,835,490000.00,Fernandópolis,Por Do Sol,NA,NA,NA
1,834,180000.00,Fernandópolis,Cohab,NA,NA,NA
2,831,250000.00,Fernandópolis,Rosa Amarela,3,3,2
3,830,180000.00,Fernandópolis,Cohab,3,3,2
4,828,140000.00,Fernandópolis,Parque Das Naçoes,3,1,2
5,826,265000.00,Fernandópolis,Fernandópolis-Sp,3,2,2
6,825,NA,Fernandópolis,Mario Benez,3,2,2
7,822,230000.00,Fernandópolis,Parque Universitario,3,2,2
8,820,170000.00,Fernandópolis,Altos Das Paineiras,2,2,2
9,818,90000.00,Fernandópolis,Araguaia,3,2,2


In [21]:
dataset.to_csv('./vendaCasas_imobiliariasp.csv', sep=';', encoding = 'utf-8-sig')